In [1]:
# GAMMA VERSION 1.0 ===========================================================
# Keep access Token
# Call out API to perform tasks
#   userId: "486e359b-20c4-4d32-bf9a-483d5c8ce3f8"
#   name: "3963436740"
#   uid: 6277729717548986270
#   email: "6277729717548986270@onuspro.exchange"
# =============================================================================
# Library for process Token
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# Library for process Datetime
from datetime import datetime, timedelta
import time
import pytz
# Library for POST GET requests
import requests
import urllib.parse
import socket
import select
import json
# import asyncio
# import httpx

# Global configuration ========================================================
access_token = None
expires_token = None
renew_needed = False
# Define the GMT+7 timezone
timezone = pytz.timezone('Asia/Bangkok')
# Check point of important handling - in UNIX timestamp
check_point = time.time()
last_order = 0
# Define your quiet hours not to send Telegram
quiet_hours = [
    ((22, 0),(23, 59)),  # 10 PM to midnight
    ((0, 0),(6, 0))      # Midnight to 6 AM
]

# Define the list of allowed coins to order ===================================
tuple_coin = (
  "UNIVNDC","CETUSVNDC","DOGEVNDC","JASMYVNDC","SOLVNDC",
  "MCAKEVNDC","SXPVNDC","CKBVNDC","EGPT1000VNDC","UXLINKVNDC",
  "MIAVNDC","MOODENGVNDC","EDU3VNDC","MEON1000VNDC","DOGSVNDC",
  "SUIVNDC","FOXYVNDC","BIGTIMEVNDC","TRUVNDC","OPVNDC",
  "LINKVNDC","XRPVNDC","1000CATVNDC","WIFVNDC","NEIROVNDC",
  "XLMVNDC","KSMVNDC","DOTVNDC","ENSVNDC","SANDVNDC",
  "GRASSVNDC","LYNXVNDC","AVAXVNDC","LISTAVNDC","TIAVNDC",
  "PEPE1000VNDC","WLDVNDC", "ADAVNDC", "HBARVNDC", "LDOVNDC"
)

# Function send to Telegram ===================================================
def send_telegram(message_string, max_retry=2):
    # Telegram bot token and chat ID
    TOKEN = "5614737400:AAHbvZrJbomt09EkpPuhadBCJl7NaGu6rlg"
    ID = "5559031253"
    # Flag to sent --> Will send if flag = 0 afterall
    flag_send = 0
    current_time = datetime.now(timezone)

    for start, end in quiet_hours:
        # print(start[0], end[0])
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        quiet_start = current_time.replace(hour=start[0],
                                           minute=start[1],
                                           second=0,
                                           microsecond=0)
        quiet_end = current_time.replace(hour=end[0],
                                         minute=end[1],
                                         second=59,
                                         microsecond=0)
        if quiet_start <= current_time <= quiet_end:
            flag_send += 1

    current_retry = 0
    while (current_retry <= max_retry) and (flag_send == 0):
        if current_retry > 0:
            print(f"Send Telegram: Retry #{current_retry}....................")
        try:
            # URL encode the message string
            encoded_message = urllib.parse.quote(message_string)
            # Construct the URL
            url1 = f"https://api.telegram.org/bot{TOKEN}/sendMessage?"
            url2 = f"chat_id={ID}&text={encoded_message}"
            # Send the request
            response = requests.get(url1 + url2, timeout=3)
            # Raise an HTTPError for bad responses (4xx and 5xx)
            response.raise_for_status()
            return True
        except requests.exceptions.RequestException as e:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, f"ERROR Telegram: {e}")
            current_retry += 1
            if current_retry > max_retry:
                print(date_time, "SEND TELEGRAM Max retries reached.")
                return False

def setup_socket(host, port):
    server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    try:
        server_socket.bind((host, port))
        server_socket.listen()
        print(f"Server listening on {host}:{port}")
        return server_socket
    except OSError as e:
        print(f"Error binding to {host}:{port} - {e}")
        server_socket.close()
        raise

# Function to log in and extract the token from cookies =======================
def get_token():
    global access_token
    global expires_token
    
    new_token = None

    while new_token is None:
        try:
            date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
            print(date_time, 'Requesting for Access Token')
            # Set up Selenium WebDriver for Microsoft Edge
            edge_options = Options()
            edge_options.add_argument("--disable-dev-shm-usage")
            edge_options.add_argument("--headless")  # Run in headless mode
            service = Service('')  # Update with your path to msedgedriver
            driver = webdriver.Edge(service=service, options=edge_options)

            driver.get("https://goonus.io/signin")
            # Perform login (update with actual login steps)
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "username"))).send_keys("0987115281")
            password_field = driver.find_element(By.ID, "password")
            password_field.send_keys("Trien825590@OU")
            password_field.send_keys(Keys.RETURN)  # Simulate pressing Enter key
            
            # Wait for login to complete
            WebDriverWait(driver, 15).until(EC.url_contains("btc_usd"))
            
            # Navigate to the desired URL
            driver.get("https://goonus.io/futures/BTC_VNDC")

            # Wait for the page to load
            time.sleep(10)  # Adjust based on actual loading time

            # Extract token from cookies
            cookies = driver.get_cookies()
            for cookie in cookies:
                # print(cookie)
                if cookie['name'] == 'onuspro-access-token':
                    new_token = cookie['value']
                    expires = cookie['expiry']
                    expires_token = datetime.fromtimestamp(int(expires), tz=pytz.utc).astimezone(timezone)
                    if new_token == access_token:
                        print("Keep same Token:", access_token[-10:])
                    else:
                        access_token = new_token
                        print("New Token:", access_token[-10:])
                        print("Token expire at:", expires_token)
                    # Close the Selenium WebDriver
                    driver.quit()
            return access_token, expires_token
        except Exception as e:
            print(e)

# Get Account Balance ==========================================================
def get_balance():
    global access_token
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/account"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "Host": "api-pro.goonus.io"
    }
    try:
        # Make the POST request with headers
        response = requests.get(url, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        index = 0
        # Collect the VNDC value
        while(response_data['assets'][index]['asset']!='VNDC'):
            index +=1
        # print(response_data['assets'][index])
        
        # Tổng tài sản
        walletBalance = float(response_data['assets'][index]['walletBalance'])
        # print("Total Balance:", walletBalance)
        # Tài sản khả dụng để order
        availableBalance = float(response_data['assets'][index]['availableBalance'])
        # print("Available:",availableBalance)
        # Tài sản đang ký quỹ
        isolatedPositionMargin = float(response_data['assets'][index]['isolatedPositionMargin'])
        # print("Isolated:", isolatedPositionMargin)
        # Tài sản đang khóa (ký quỹ + đang order)
        orderMargin = float(response_data['assets'][index]['orderMargin'])
        orderFee = float(response_data['assets'][index]['orderFee'])
        lockedBalance = isolatedPositionMargin + orderMargin + orderFee
        # print("Locked:", lockedBalance)
        return walletBalance,availableBalance,lockedBalance

    except requests.RequestException as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time}: Error Get balance {e}")
        return None,None,None

# Get Open Position list =======================================================
def get_position():
    global access_token
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/positions?status=OPEN"
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
        "Host": "api-pro.goonus.io"
    }
    try:
        # Make the POST request with headers
        response = requests.get(url, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        list_position = []
        for i in range(len(response_data)):
            # print(response_data[i]["symbol"])
            list_position.append(response_data[i]["symbol"])        
        return list_position
    except requests.RequestException as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time}: Error Get balance {e}")
        return []

# Set Leverage =================================================================
def set_leverage(coin_name, leverage_number):
    global access_token
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/leverage"
    data = {"symbol": coin_name, "leverage": leverage_number}
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    try:
        # Make the POST request with headers
        response = requests.post(url, json=data, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['userId']=='486e359b-20c4-4d32-bf9a-483d5c8ce3f8'):
            print(f'Done set leverage for {coin_name} at x{leverage_number}')
        # if response_data['userId']

    except requests.RequestException as e:
        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
        print(f"{date_time}: Error set Leverage {e}")

# Place Strategic Order =======================================================
def set_strategy(coin_name, price, amount, profit, loss):
    global access_token
    url = "https://api-pro.goonus.io/perpetual/v1/strategyOrder"
    payload = { "orders": [
        {
        "symbol": coin_name,
        "side": "BUY",
        "type": "MARKET",
        "positionSide": "BOTH",
        "price": price,
        "size": amount,
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "workingType": "MARK_PRICE",
        "stopPrice": "0",
        "reduceOnly": False,
        "timeInForce": "GTC",
        "closePosition": False
        },
        {
        "type": "TAKE_PROFIT",
        "price": 0,
        "stopPrice": profit,
        "workingType": "CONTRACT_PRICE",
        "symbol": coin_name,
        "side": "SELL",
        "size": amount,
        "positionSide": "BOTH",
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "reduceOnly": True,
        "closePosition": False
        },
        {
        "type": "STOP",
        "price": 0,
        "stopPrice": loss,
        "workingType": "CONTRACT_PRICE",
        "symbol": coin_name,
        "side": "SELL",
        "size": amount,
        "positionSide": "BOTH",
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "reduceOnly": True,
        "closePosition": False
        }
    ]}
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    # print(payload)
    try:
        # Make the POST request with headers
        response = requests.post(url, json=payload, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['status'] == 'WORKING'):
            print(f'Successfull place Strategic order {coin_name} ')
            return True
        else:
            print(f'Error: {response_data}')
            return False
    except requests.RequestException as e:
        print(f"{e}")
        return False

# Place Order =================================================================
def set_order(coin_name, price, amount):
    global access_token
    url = "https://api-pro.goonus.io/perpetual/v1/order"
    data = {
        "symbol": coin_name,
        "side": "BUY",
        "type": "MARKET",
        "positionSide": "BOTH",
        "price": price,
        "size": amount,
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "workingType": "MARK_PRICE",
        "stopPrice": "0",
        "reduceOnly": False,
        "timeInForce": "GTC",
        "closePosition": False
    }
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json"
    }
    try:
        # Make the POST request with headers
        response = requests.post(url, json=data, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['userId'] == '486e359b-20c4-4d32-bf9a-483d5c8ce3f8'):
            print(f'Successfull place order {coin_name} ')
        else:
            print(f'Error: {response_data}')
    except requests.RequestException as e:
        print(f"{e}")

# Close Order =================================================================
def close_order(coin_name, amount):
    # Define the API endpoint and parameters
    url = "https://api-pro.goonus.io/perpetual/v1/order"
    data = {
        "symbol": coin_name,
        "side": "SELL",
        "type": "MARKET",
        "positionSide": "BOTH",
        "price": 0,
        "size": amount,
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "workingType": "MARK_PRICE",
        "stopPrice": 0,
        "reduceOnly": True,
        "id": 767072
    }

    headers = {
        "Authorization": f"Bearer {TOKEN2}",
        "Content-Type": "application/json"
    }
    try:
        # Make the POST request with headers
        response = requests.post(url, json=data, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['userId'] == '486e359b-20c4-4d32-bf9a-483d5c8ce3f8'):
            print(f'Successfull close order for {coin_name}')
        else:
            print(f'Error: {response_data}')
    except requests.RequestException as e:
        print(f"{e}")

def set_profit(coin_name, price):
    url = "https://api-pro.goonus.io/perpetual/v1/order"
    data = {
        "symbol": coin_name,
        "side": "SELL",
        "type": "TAKE_PROFIT",
        "positionSide": "BOTH",
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "reduceOnly": False,
        "closePosition": True,
        "price": 0,
        "stopPrice": price,
        "workingType": "CONTRACT_PRICE"
    }
    headers = {
        "Authorization": f"Bearer {TOKEN2}",
        "Content-Type": "application/json"
    }
    try:
        # Make the POST request with headers
        response = requests.post(url, json=data, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['userId'] == '486e359b-20c4-4d32-bf9a-483d5c8ce3f8'):
            print(f'Successfull set Profit {coin_name} ')
        else:
            print(f'Error: {response_data}')
    except requests.RequestException as e:
        print(f"{e}")

def set_loss(coin_name, price):
    url = "https://api-pro.goonus.io/perpetual/v1/order"
    data = {
        "symbol": coin_name,
        "side": "SELL",
        "type": "STOP",
        "positionSide": "BOTH",
        "clientOrderId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "userId": "486e359b-20c4-4d32-bf9a-483d5c8ce3f8",
        "postOnly": False,
        "timeInForce": "GTC",
        "reduceOnly": False,
        "closePosition": True,
        "price": 0,
        "stopPrice": price,
        "workingType": "CONTRACT_PRICE"

    }
    headers = {
        "Authorization": f"Bearer {TOKEN2}",
        "Content-Type": "application/json"
    }
    try:
        # Make the POST request with headers
        response = requests.post(url, json=data, headers=headers, timeout=3)
        # Raise an HTTPError for bad responses (4xx and 5xx)
        response.raise_for_status()
        # Parse the JSON response
        response_data = response.json()
        if (response_data['userId'] == '486e359b-20c4-4d32-bf9a-483d5c8ce3f8'):
            print(f'Successfull set Loss {coin_name} ')
        else:
            print(f'Error: {response_data}')
    except requests.RequestException as e:
        print(f"{e}")


# Run the main function =======================================================
if __name__ == "__main__":
    date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
    message = f"{date_time}: Gamma started"
    print(message)
    send_telegram(message)

    # Process Login and get the access token ==================================
    get_token()

    # Config for Server socket ================================================
    server_socket = setup_socket('localhost', 65432)
    inputs = [server_socket]
    
    try:        
        # Infinite loop to execute
        while True:
            # Monitors the sockets in the inputs list for readability.
            # It returns three lists: readable, writable, and exceptional.
            # Here, we only care about readable with Timeout 1 second
            readable, _, _ = select.select(inputs, [], [], 1)
            # Iterates over the sockets that are ready for reading
            for s in readable:
                if s is server_socket:
                    conn, addr = server_socket.accept()
                    inputs.append(conn)
                    # print(f"Accepted connection from {addr}")
                else:
                    # Reads up to 1024 bytes of data from the socket.
                    data = s.recv(1024)
                    # Checks if data was received
                    if data:
                        order_info = json.loads(data.decode('utf-8'))
                        # START PROCESS =================================================
                        # {"coin_name": "BTCVNDC","order_price": 123.1}
                        date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                        print(date_time,'RECEIVED -->', order_info)
                        # Double check token
                        if access_token is None:
                            get_token()
                        # Check if coin is allowed & avoid duplicate order
                        exclude_list = get_position()
                        if (order_info['coin_name'] in tuple_coin) and (order_info['coin_name'] not in exclude_list) and (time.time() - last_order > 300):
                            # Get current available balance
                            total,available,locked = get_balance()
                            if total is not None:
                                # print(total,available,locked)
                                # Process choose the leverage --> variance but default x5
                                leverage = 10
                                set_leverage(order_info['coin_name'], leverage)
                                # Choose the amount VNDC of investment
                                # Idea: 20% of Total balance < availabe balane
                                # Profit 2.5% , Loss 10%
                                order_amount = total/20
                                order_price = order_info['order_price']
                                amount = round((leverage * order_amount)/order_price , 4)
                                profit = order_price * (1 + 4/leverage/100)
                                loss = order_price * (1 - 12/leverage/100)
                                if order_amount < available:
                                    # Place order
                                    # print("Order Amount:\t",order_amount)
                                    # print("Order Price:\t", order_price)
                                    # print("Amount:\t",amount)
                                    # print("Profit:\t", profit)
                                    # print("Loss:\t",loss)
                                    if set_strategy(order_info['coin_name'], order_price, amount, profit, loss):
                                        send_telegram(f"Done Order {order_info['coin_name']}")
                                        last_order = time.time()
                                else:
                                    print(f"Order {order_info['coin_name']}: Balance not sufficient")

                    else:
                        # Removes the socket from the inputs list.
                        inputs.remove(s)
                        s.close()

            if (time.time() - check_point > 300 ):
                date_time = datetime.now(timezone).strftime('%Y-%m-%d %H:%M:%S')
                print(date_time, "Process checkpoint")
                # Process every 10min checkpoint to record the expired time to renew the token
                try:
                    delta_time = expires_token - datetime.now(timezone)
                    print(get_position())
                    if delta_time < timedelta(minutes=10):
                        print(date_time,'Renew the token')
                        get_token()
                        send_telegram('Gamma now renew Token')
                    else:
                        print(date_time,'Token still valid')            
                    check_point = time.time()
                except Exception as e:
                    print(f"Error Checkpoint: {e}")


    except KeyboardInterrupt:
        print("Server is shutting down...")
    finally:
        server_socket.close()    

    print(f"{date_time}: Stop Processing")
    # set_leverage("BTCVNDC", 5)


2024-12-02 00:01:20: Gamma started
2024-12-02 00:01:20 Requesting for Access Token
New Token: gltbWTHExw
Token expire at: 2024-12-02 01:11:41+07:00
Server listening on localhost:65432
Server is shutting down...
2024-12-02 00:01:20: Stop Processing
